# Relation Prediction in Argument Mining with Pre-trained Deep Bidirectional Transformers

Code for BA thesis.

In [9]:
# All imports
# import os 
# import re

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold


from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
from datetime import datetime

In [10]:
def load_local_data(filename, data):
    df = pd.read_csv(filename, sep='\t')
    print(df.groupby('org_dataset').org.apply(lambda x: x.str.split().str.len().mean()))
    print(df.groupby('org_dataset').response.apply(lambda x: x.str.split().str.len().mean()))
    # Split in Training and Validation data
    if data == 'node':
        # Training data: NoDe debatepedia all versions without neutral label
        # Validation data: NoDe procon
        dataset = df.loc[~df['org_dataset'].isin(['political', 'comargGM', 'comargUGIP', 'agreement'])]
        dataset = df.loc[df['org_dataset'].isin(['debate_test', 'debate_train', 'procon'])] # Use orignal data
        # dataset = dataset[dataset['label'] != 'unrelated'] # Filter only support/attack
        dataset = dataset.sample(frac=1)
        #data_train = dataset.iloc[:-100]
        #data_val = dataset #.iloc[-100:]
        data_train = dataset.loc[~dataset['org_dataset'].isin(['debate_test'])]
        data_val = dataset.loc[dataset['org_dataset'].isin(['debate_test'])]
    elif data == 'political':
        dataset = df.loc[df['org_dataset'].isin(['political'])]
        #dataset = dataset[dataset['label'] != 'unrelated'] # Filter only support/attack
        dataset = dataset.sample(frac=1)
        data_train = dataset.iloc[:-200]
        data_val = dataset.iloc[-200:]
    elif data == 'agreement':
        dataset = df.loc[df['org_dataset'].isin(['agreement'])]
        dataset = dataset.sample(frac=1).dropna()
        data_train = dataset.iloc[:-2000]
        data_val = dataset.iloc[-2000:]
    else:
        print('Invalid dataset')
        sys.exit(-1)
    return data_train, data_val

def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""
    
    is_training = not is_predicting
    
    """
    model = modeling.BertModel(
        config=BERT_CONFIG,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=False)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = model.get_pooled_output()
    
    """
    tags = set()
    if is_training:
        tags.add("train")
    bert_module = hub.Module(
        BERT_MODEL_HUB,
        tags=tags,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)
    

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]
    
    #######
    

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # Dropout helps prevent overfitting
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN 
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = optimization.create_optimizer(
              loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)


            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                  loss=loss,
                  train_op=train_op)

        # TEST
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn   

In [11]:
# All parameters
cross_val = True
data_path = '../data/complete_data.tsv'
dataset = 'political'  # One of 'agreement', 'node' and 'political'
use_org = True
use_resp = True
convert_dicts = {'agreement': {"agreement": 0, "disagreement": 1, "unrelated": 2},
                'node': {"attack": 0, "support": 1, "unrelated": 2},
                'political': {"attack": 0, "support": 0, "unrelated": 1}}
                #'political': {"attack": 0, "support": 1, "unrelated": 1}}
convert_dict = convert_dicts[dataset]

ORG_COLUMN = 'org'
RESP_COLUMN = 'response'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1] # [0, 1, 2]
BERT_VOCAB= './uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = './uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG_PATH = './uncased_L-12_H-768_A-12/bert_config.json'
BERT_CONFIG = modeling.BertConfig.from_json_file(BERT_CONFIG_PATH)
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"



MAX_SEQ_LENGTH = 128

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 5000
SAVE_SUMMARY_STEPS = 100
OUTPUT_DIR = 'BERT_RUN' + str(datetime.now()) + dataset

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
#run_config = tf.contrib.tpu.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)


In [12]:
# Create tokenizer
tokenization.validate_case_matches_checkpoint(True, BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [13]:
# Load local data
train_df, test_df = load_local_data(data_path, dataset)
print(train_df.head())

# Create datasets (Only take up to max_seq_length words for memory)

train_df = train_df.replace({'label': convert_dict})
test_df = test_df.replace({'label': convert_dict})
print(train_df.groupby('label').describe())
print(test_df.groupby('label').describe())
train = train_df.sample(frac=1)
test = test_df.sample(frac=1)

# Use org + response
if use_org and use_resp:
    # Use the InputExample class from BERT's run_classifier code to create examples from the data
    train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                       text_a = x[ORG_COLUMN], 
                                                                       text_b = x[RESP_COLUMN], 
                                                                       label = x[LABEL_COLUMN]), axis = 1)

    test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                       text_a = x[ORG_COLUMN], 
                                                                       text_b = x[RESP_COLUMN], 
                                                                       label = x[LABEL_COLUMN]), axis = 1)
# Use only org
elif use_org:
    # Use the InputExample class from BERT's run_classifier code to create examples from the data
    train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                       text_a = x[ORG_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)

    test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                       text_a = x[ORG_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)
# Use only resp
else:
    # Use the InputExample class from BERT's run_classifier code to create examples from the data
    train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                       text_a = x[RESP_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)

    test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                       text_a = x[RESP_COLUMN], 
                                                                       text_b = None, 
                                                                       label = x[LABEL_COLUMN]), axis = 1)
    
# Convert our train and test features to InputFeatures that BERT understands.
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

org_dataset
agreement              74.945753
comargGM               13.558755
comargUGIP              9.486594
debate_ext_attacks     36.043478
debate_ext_media       53.533333
debate_ext_second      11.043478
debate_ext_supp        18.885714
debate_extended        35.772358
debate_test            13.860000
debate_train           17.280000
political             102.759589
procon                 13.950000
Name: org, dtype: float64
org_dataset
agreement              69.926330
comargGM              115.659922
comargUGIP             83.567031
debate_ext_attacks     32.782609
debate_ext_media       53.168254
debate_ext_second      60.717391
debate_ext_supp        45.428571
debate_extended        59.934959
debate_test            51.240000
debate_train           48.080000
political             102.171233
procon                 30.366667
Name: response, dtype: float64
     org_dataset    id                                                org  \
4075   political  1280   What are the folks going 

I0620 08:25:40.542505 139737458554624 tf_logging.py:115] Writing example 0 of 1260


INFO:tensorflow:*** Example ***


I0620 08:25:40.545546 139737458554624 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0620 08:25:43.284555 139737458554624 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


I0620 08:25:43.285073 139737458554624 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:label: 0 (id = 0)


I0620 08:25:43.285549 139737458554624 tf_logging.py:115] label: 0 (id = 0)


In [14]:
if not cross_val:
    # Compute # train and warmup steps from batch size
    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    model_fn = model_fn_builder(
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    """model_fn = run_classifier.model_fn_builder(
        bert_config=BERT_CONFIG,
        num_labels=len(label_list),
        init_checkpoint=False,
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=False,
        use_one_hot_embeddings=False)"""

    estimator = tf.estimator.Estimator(
    #estimator = tf.contrib.tpu.TPUEstimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
      #train_batch_size=BATCH_SIZE)

    # Create an input function for training. drop_remainder = True for using TPUs.
    train_input_fn = run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    print(f'Beginning Training!')
    current_time = datetime.now()
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Training took time ", datetime.now() - current_time)
    
    test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
    
    predictions = estimator.predict(input_fn=test_input_fn)
    pred_label = [prediction['labels'] for prediction in predictions]
    print("Confusion Matrix:")
    print(confusion_matrix(test['label'].values.astype(int), np.array(pred_label)))
    print("Classification Report:")
    if len(set(test['label'])) == 3:
        print(classification_report(test['label'].values.astype(int), pred_label, target_names=["attack", "support", "unrelated"]))
    else: 
        #print(classification_report(test['label'].values.astype(int), pred_label, target_names=["agreement", "disagreement"]))
        print(classification_report(test['label'].values.astype(int), pred_label, target_names=["attack", "support"]))
        #print(classification_report(test['label'].values.astype(int), pred_label, target_names=["relation", "unrelated"]))

In [15]:
def run_training(train_features, test_features, test, train, target_names):
    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
    model_fn = model_fn_builder(
        num_labels=len(label_list),
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps)
    train_input_fn = run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)
    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)
    print(f'Beginning Training!')
    print('test labels', test['label'].value_counts())
    print('train labels', train['label'].value_counts())
    current_time = datetime.now()
    estimator = tf.estimator.Estimator(
              model_fn=model_fn,
              config=run_config,
              params={"batch_size": BATCH_SIZE})
    estimator = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    predictions = estimator.predict(input_fn=test_input_fn, yield_single_examples=False)
    print(predictions)
    pred_label = np.array([])
    pred_label = [np.append(pred_label, prediction['labels']) for prediction in predictions]
    pred_label = np.concatenate(pred_label).ravel()
    print(pred_label)
    class_rep = classification_report(test['label'].values.astype(int), pred_label, labels= [0,1], target_names=target_names, output_dict=True)
    acc = accuracy_score(test['label'].values.astype(int), pred_label)
    print(classification_report(test['label'].values.astype(int), pred_label, target_names=target_names))
    print("Accuracy:",  acc)
    print("Training took time ", datetime.now() - current_time)
    return (class_rep, acc)

def cross_validate(data):
    results = []
    skf = StratifiedKFold(n_splits=10)
    for train_idx, val_idx in skf.split(data, data['label']):
        try:
            tf.gfile.DeleteRecursively(OUTPUT_DIR)
        except:
            # Doesn't matter if the directory didn't exist
            pass
        tf.gfile.MakeDirs(OUTPUT_DIR)
        train = data.iloc[train_idx]
        test = data.iloc[val_idx]
        print(train.shape, test.shape)
        # Use the InputExample class from BERT's run_classifier code to create examples from the data
        train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                           text_a = x[ORG_COLUMN], 
                                                                           text_b = x[RESP_COLUMN], 
                                                                           label = x[LABEL_COLUMN]), axis = 1)

        test_InputExamples = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                           text_a = x[ORG_COLUMN], 
                                                                           text_b = x[RESP_COLUMN], 
                                                                           label = x[LABEL_COLUMN]), axis = 1)

        # Convert our train and test features to InputFeatures that BERT understands.
        train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
        test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
        results.append(run_training(train_features, test_features, test, train, ["relation", "unrelated"]))
    return results

In [16]:
if cross_val:

    result = cross_validate(train_df.append(test_df))

(1313, 8) (147, 8)
INFO:tensorflow:Writing example 0 of 1313


I0620 08:25:43.761950 139737458554624 tf_logging.py:115] Writing example 0 of 1313


INFO:tensorflow:*** Example ***


I0620 08:25:43.765559 139737458554624 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:Running local_init_op.


I0620 09:01:33.179440 139737458554624 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0620 09:01:33.220756 139737458554624 tf_logging.py:115] Done running local_init_op.


[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1.
 1.]
              precision    recall  f1-score   support

    relation       0.67      0.68      0.68        73
   unrelated       0.67      0.65      0.66        72

    accuracy                           0.67       145
   macro avg       0.67      0.67      0.67       145
weighted avg       0.67      0.67      0.67       145

Accuracy: 0.6689655172413793
Training took time  0:03:33.459099


In [17]:
if cross_val:
    #print(result)
    res = np.array(result)[:,0]
    result_df = pd.DataFrame(res[0])
    for data in res[1:]:
        new_data = pd.DataFrame(data)
        result_df = result_df.append(new_data)
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    print(result_df.groupby(level=0).agg([np.mean, np.max, np.min]).transpose())

                   f1-score  precision    recall     support
relation     mean  0.654506   0.640441  0.671566   73.100000
             amax  0.727273   0.742857  0.756757   74.000000
             amin  0.542857   0.552941  0.520548   73.000000
unrelated    mean  0.633804   0.652457  0.618702   72.900000
             amax  0.738255   0.723684  0.753425   73.000000
             amin  0.522388   0.556962  0.479452   72.000000
accuracy     mean  0.645188   0.645188  0.645188    0.645188
             amax  0.732877   0.732877  0.732877    0.732877
             amin  0.561644   0.561644  0.561644    0.561644
macro avg    mean  0.644155   0.646449  0.645134  146.000000
             amax  0.732764   0.733271  0.732877  147.000000
             amin  0.558662   0.562063  0.561644  145.000000
weighted avg mean  0.644174   0.646433  0.645188  146.000000
             amax  0.732764   0.733271  0.732877  147.000000
             amin  0.558662   0.562063  0.561644  145.000000


In [18]:
#print(estimator.get_variable_names())